In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
import pandas as pd
from stop_words import stop_words
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# nltk.download("punkt") 

## PREPROCESSING

#### 1. TOKENISASI
#### 2. STEMMING
#### 3. STOP WORDS REMOVAL

In [12]:
stemmer = PorterStemmer()
label_encoder = LabelEncoder()

In [82]:
def steaming(word):
    stemmer = PorterStemmer()
    stemmed_word = stemmer.stem(word)
    return stemmed_word

In [83]:
def preprocessing(text):
    # Menghilangkan tanda baca
    text = re.sub(r'[^\w\s-]', ' ', text)

    # Memisahkan teks menjadi kata-kata
    words = text.split()
    print(words)
    
    # Menghilangkan stopwords
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Menghilangkan singkatan sederhana
    filtered_words = [re.sub(r'\.', '', word) for word in filtered_words]

    # Menghilangkan angka
    filtered_words = [re.sub(r'\d', '', word) for word in filtered_words]
    
    filtered_words = set(filtered_words)
    
    # STEAMING
    filtered_words = [steaming(i) for i in filtered_words]

    return ' '.join(sorted(filtered_words))


In [75]:
json_data = pd.read_json("./dataset/intents.json")
df = pd.json_normalize(json_data['intents'])
df = df.drop('context_set', axis=1)

In [76]:
df['tag']

0           greeting
1            goodbye
2            creator
3               name
4              hours
5             number
6             course
7               fees
8           location
9             hostel
10             event
11          document
12            floors
13          syllabus
14           library
15    infrastructure
16           canteen
17              menu
18         placement
19             ithod
20       computerhod
21           extchod
22         principal
23               sem
24         admission
25       scholarship
26        facilities
27    college intake
28           uniform
29         committee
30            random
31             swear
32          vacation
33            sports
34         salutaion
35              task
36           ragging
37               hod
Name: tag, dtype: object

In [77]:
df['patterns'] = df['patterns'].apply(lambda x : ' '.join(x))

In [79]:
df

,tag,patterns,responses
0,greeting,Hi How are you? Is anyone there? Hello Good da...,"[Hello!, Good to see you again!, Hi there, how..."
1,goodbye,cya see you bye bye See you later Goodbye I am...,"[Sad to see you go :(, Talk to you later, Good..."
2,creator,what is the name of your developers what is th...,[College students]
3,name,name your name do you have a name what are you...,"[You can call me Mind Reader., I'm Mind Reader..."
4,hours,timing of college what is college timing worki...,[College is open 8am-5pm Monday-Saturday!]
5,number,more info contact info how to contact college ...,[You can contact at: NUMBER]
6,course,list of courses list of courses offered list o...,"[Our university offers Information Technology,..."
7,fees,information about fee information on fee tell ...,"[For Fee detail visit <a target=""_blank"" href=..."
8,location,where is the college located college is locate...,"[<a target=""_blank"" href=""ADD YOU GOOGLE MAP L..."
9,hostel,hostel facility hostel servive hostel location...,"[For hostel detail visit <a target=""_blank"" hr..."


In [84]:
patterns = df['patterns'].apply(preprocessing)
encoded_labels = label_encoder.fit_transform(df['tag'])

['Hi', 'How', 'are', 'you', 'Is', 'anyone', 'there', 'Hello', 'Good', 'day', 'What', 's', 'up', 'how', 'are', 'ya', 'heyy', 'whatsup']
['cya', 'see', 'you', 'bye', 'bye', 'See', 'you', 'later', 'Goodbye', 'I', 'am', 'Leaving', 'Bye', 'Have', 'a', 'Good', 'day', 'talk', 'to', 'you', 'later', 'ttyl', 'i', 'got', 'to', 'go', 'gtg']
['what', 'is', 'the', 'name', 'of', 'your', 'developers', 'what', 'is', 'the', 'name', 'of', 'your', 'creators', 'what', 'is', 'the', 'name', 'of', 'the', 'developers', 'what', 'is', 'the', 'name', 'of', 'the', 'creators', 'who', 'created', 'you', 'your', 'developers', 'your', 'creators', 'who', 'are', 'your', 'developers', 'developers', 'you', 'are', 'made', 'by', 'you', 'are', 'made', 'by', 'whom', 'who', 'created', 'you', 'who', 'create', 'you', 'creators', 'who', 'made', 'you', 'who', 'designed', 'you']
['name', 'your', 'name', 'do', 'you', 'have', 'a', 'name', 'what', 'are', 'you', 'called', 'what', 'is', 'your', 'name', 'what', 'should', 'I', 'call', 'you

In [80]:
patterns = df['patterns'].apply(preprocessing)

NameError: name 'preprocessing' is not defined

In [74]:
print(' '.join(patterns.tolist()).split())

NameError: name 'patterns' is not defined

## TF-IDF

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# create tf-idf
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(patterns)

# print idf
print(vectorizer.idf_)

In [ ]:
df_tfidf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

In [ ]:
df_tfidf.shape

### DATA PREPARATION

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_tfidf, encoded_labels, test_size=0.10, random_state=42)
display(X_train, y_train)

In [ ]:
# NORMALISASI
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
norm_X_train = scaler.fit_transform(X_train)
norm_X_test = scaler.fit_transform(X_test)

In [ ]:
display(norm_X_train, norm_X_test)

### WORD EMBEDING

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [54]:
X_train, X_test, y_train, y_test = train_test_split(df['patterns'].apply(lambda x : ''.join(x)), df['tag'], test_size=0.1)
display(len(X_train),X_test,len(y_train),y_test)

34

15    how is college infrastructure infrastructure c...
37                          hod hod name who is the hod
13    Syllabus for IT what is the Information Techno...
35    what can you do what are the thing you can do ...
Name: patterns, dtype: object

34

15    infrastructure
37               hod
13          syllabus
35              task
Name: tag, dtype: object

In [44]:
maxlen = 100
vocab_length = 10000

In [45]:
# Tokenize the text data
tokenizer = Tokenizer(num_words=vocab_length) # num_word membatasi jumlah kata unik yang akan diperhitungkan oleh Tokenizer.
tokenizer.fit_on_texts(X_train) # Membangun kamus kata-kata berdasarkan dataset latih X_train. Setiap kata dalam dataset akan mendapatkan indeks numerik sesuai dengan kamus yang dibangun oleh Tokenizer. 
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test) 

In [48]:
# Encode the sentiment labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

In [49]:
X_train_padded = pad_sequences(X_train_sequences, maxlen=maxlen) 
X_test_padded = pad_sequences(X_test_sequences, maxlen=maxlen)

In [56]:
# Model definition
model = Sequential()

# Embedding layer
model.add(Embedding(vocab_length, 100, input_length=maxlen, trainable=False))

# Convolutional Layer 1
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(2))

# Convolutional Layer 2
model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(2))

# Global Max Pooling
model.add(GlobalMaxPooling1D())

# Dense layers
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='sigmoid'))  # Binary classification

In [57]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [58]:
history = model.fit(X_train_padded, y_train_encoded, epochs=100, batch_size=100, validation_data=(X_test_padded, y_test_encoded))

Epoch 1/100
1/1 [==============================] - 1s 726ms/step - loss: 0.3878 - accuracy: 0.0588 - val_loss: 0.6004 - val_accuracy: 0.2500
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: -1.1637 - accuracy: 0.0294 - val_loss: 0.5152 - val_accuracy: 0.2500
Epoch 3/100
1/1 [==============================] - 0s 27ms/step - loss: -2.5967 - accuracy: 0.0294 - val_loss: 0.4271 - val_accuracy: 0.2500
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: -4.4629 - accuracy: 0.0294 - val_loss: 0.3334 - val_accuracy: 0.2500
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: -6.0881 - accuracy: 0.0294 - val_loss: 0.2327 - val_accuracy: 0.2500
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: -7.9979 - accuracy: 0.0294 - val_loss: 0.1219 - val_accuracy: 0.2500
Epoch 7/100
1/1 [==============================] - 0s 26ms/step - loss: -9.5574 - accuracy: 0.0294 - val_loss: -1.4105e-04 - val_accuracy: 0.2500
Epoch 8/

In [59]:
loss, accuracy = model.evaluate(X_test_padded, y_test_encoded)

1/1 [==============================] - 0s 18ms/step - loss: -6130.4219 - accuracy: 0.2500


## TRAINING MODEL

### NEURAL NETWORK

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

In [ ]:
# Membuat model NN
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(set(encoded_labels)), activation='softmax'))

# Kompilasi model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Melatih model
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

In [ ]:
loss, accuracy = model.evaluate(norm_X_test, y_test)

## SVM

In [61]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [85]:
# Membuat model SVM dengan TF-IDF Vectorizer
svm_model = make_pipeline(TfidfVectorizer(), SVC(kernel='rbf'))

# Melatih model SVM
svm_model.fit(X_train, y_train_encoded)

# Membuat prediksi
svm_predictions = svm_model.predict(X_test)

# Menghitung akurasi
accuracy = accuracy_score(y_test_encoded, svm_predictions)
print("Akurasi SVM:", accuracy)

Akurasi SVM: 0.0


In [86]:
display(y_test_encoded, svm_predictions)

array([0, 2, 1, 3])

array([23, 24, 24, 12])

In [87]:
# Langkah 1: Preprocessing Data Baru
new_data = [
        'Hi there! How can I help you?',
        'What courses are available at the university?',
        'How can I apply for admission?',
        'Tell me about the campus facilities.',
        'Are there any upcoming events?',
        'How can I contact the university?',"is there any canteen",   
        "Who is the head of the Information Technology department?",
        "Who is the head of the Computer Science department?",
    ]

# Lakukan TF-IDF pada data baru
new_data_vectorized = vectorizer.transform(new_data)

# Langkah 3: Prediksi
predictions = model.predict(new_data_vectorized)

NameError: name 'vectorizer' is not defined

In [ ]:
predictions = model.predict(new_data_vectorized)

# Ambil label sebelum diencoded
predicted_labels = [df['tag'].iloc[prediction.argmax()] for prediction in predictions]

# Tampilkan hasil prediksi
for i, data in enumerate(new_data):
    print(f"Data: {data}, Predicted Label: {df['responses'].loc[df['tag'] == predicted_labels[i]].tolist()[0][0]}") 